# Tutorial 1 - Spike Encoding

## Learn how to
- convert datasets into spiking datasets
- how to visualize them
- how to generate random spike trains

> MNIST 데이터셋을 SNN으로 학습하기 위한 spiking dataset으로 변환하고 사용하는 방법에 대해 공부한다.

---

# 1. Setting up the MNIST dataset

In [11]:
import snntorch as snn
import torch

# Traning params
batch_size  = 128
data_path   = './data'
num_classes = 10    # MNIST 0~9

# Torch variables
dtype = torch.float

In [16]:
# Download Dataset
from torchvision import datasets, transforms

# Define a transform
transforms = transforms.Compose([
             transforms.Resize((28,28)),
             transforms.Grayscale(),
             transforms.ToTensor(),
             transforms.Normalize((0,), (1,))
             ])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transforms)

MNIST 데이터셋은 60,000장의 이미지를 가지고 있다. 빠른 구현을 위해 서브셋으로 전체 데이터의 1/10인 6,000장의 데이터셋을 생성해준다.

snntorch.utils는 서브셋을 생성해주는 함수를 포함한다.

In [17]:
from snntorch import utils

subset_size = 10
mnist_train = utils.data_subset(mnist_train, subset_size)

print(f'The size of mnist_train is {len(mnist_train)}')

The size of mnist_train is 6000


위에서 가져온 데이터셋은 메모리에 저장된다. PyTorch의 DataLoader는 이 데이터셋을 배치 단위로 네트워크에 제공해준다.

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)

# 2. Spike Encoding

SNN의 입력으로 데이터를 제공하기 위해 두 가지 방식을 고려할 수 있다. 하나는 동일한 이미지를 매 시간(time step)마다 제공하는 것이고, 다른 하나는 시간에 의존하는(time-varying) spikes(sequence)로 변환하여 제공하는 것이다. 두 번째 방식이 SNN에 더 적합하므로 해당 방식에 대해 알아보겠다.